In [1]:
import pandas as pd
import numpy as np

import sqlite3
import sqlalchemy as sqa
 
import plotly_express as px

In [2]:
class db(): pass

In [3]:
db.base_path = '/sfs/qumulo/qhome/rca2t/Public/CAPSTONES/law-as-data'
db.db_file = f'{db.base_path}/db/ussc.db'
db.OHCO = ['vol_num','case_num','position']
db.dbconn = sqa.create_engine(f'sqlite:///{db.db_file}')

In [4]:
db.table_names = sqa.inspect(db.dbconn).get_table_names()

In [5]:
print('\n'.join(db.table_names))

AUTHOR
BOW
CASE
CORPUS
CORPUS_COMPRESSED
D2V
D2VP
THETA
THETA_NMF
TOPICS
TOPICS_NMF
VOCAB
YEAR


In [7]:
def import_table(self, table_name, table_index=None):
    df = pd.read_sql(table_name, self.dbconn) 
    if table_index:
        df = df.set_index(table_index).sort_index()
    setattr(self, table_name, df)

db.import_table = import_table.__get__(db)

In [8]:
db.import_table('CORPUS_COMPRESSED', db.OHCO)

In [9]:
db.CORPUS_COMPRESSED

doc_content  \
vol_num case_num position                                                      
3       171      dissent   DISSENT BY: CUSHING\nCUSHING, Justice. As I ha...   
                 opinion   THE COURT delivered their opinions seriatim in...   
        321      dissent   DISSENT BY: WILSON\nWILSON, Justice. I conside...   
                 opinion   ELSWORTH, Chief Justice. The question, how far...   
        386      dissent   DISSENT BY: IREDELL\nIREDELL, Justice. Though ...   
...                                                                      ...   
554     471      opinion   Justice Souter delivered the opinion of the Co...   
        527      dissent   DISSENT BY: Stevens \nDISSENT \nJustice Steven...   
                 opinion   Justice Scalia delivered the opinion of the Co...   
        570      dissent   Justice Breyer, with whom Justice Stevens, Jus...   
                 opinion   Justice Scalia delivered the opinion of the Co...   

                           doc_len  
vol_num case_num position           
3       171      dissent       252  
                 opinion      7826  
        321      dissent     16365  
                 opinion       824  
        386      dissent      8908  
...                            ...  
554     471      opinion     63276  
        527      dissent     25244  
                 opinion     45723  
        570      dissent    150367  
                 opinion    107923  

[12348 rows x 2 columns]

In [10]:
import spacy

In [11]:
from tqdm.auto import tqdm
tqdm.pandas()

In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
doc = nlp(db.CORPUS_COMPRESSED.sample().doc_content.values[0])

In [ ]:
db.CORPUS_COMPRESSED.doc_content.progress_apply(lambda x: nlp(x), 1)

In [15]:
# for entity in doc.ents:
#     print(entity.text, entity.label_)

In [16]:
# for token in doc:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#             token.shape_, token.is_alpha, token.is_stop)

In [18]:
# POS = [(token.text, token.lemma_, token.pos_) for token in doc

In [19]:
# POS

In [20]:
import spacy
from spacy.tokens import DocBin

In [26]:
texts = db.CORPUS_COMPRESSED.doc_content.values

In [ ]:
doc_bin = DocBin(attrs=["LEMMA", "ENT_IOB", "ENT_TYPE"], store_user_data=True)
for doc in nlp.pipe(texts):
    doc_bin.add(doc)
bytes_data = doc_bin.to_bytes()

In [27]:
# Deserialize later, e.g. in a new process
nlp = spacy.blank("en")
doc_bin = DocBin().from_bytes(bytes_data)
docs = list(doc_bin.get_docs(nlp.vocab))

KeyboardInterrupt: 